In [1]:
import pandas as pd

# 1.加载数据：

In [2]:
# 加载训练数据集
train = pd.read_csv('./data/titanic_train.csv')

In [3]:
# 加载测试数据集
test = pd.read_csv('./data/titanic_test.csv')

In [4]:
train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


# 2.数据预处理：

### 像statsmodels和scikit-learn这样的库一般不能输入缺失数据，所以需要检查各列，看看是否包含缺失数据。

In [5]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

### 如果需要使用 Age 作为预测因子，但是它有缺失数据。因此首先需要进行缺失数据插补（imputation），本次仅做一个简单的插补——用训练数据集的中位数来填补空值。

In [7]:
impute_value = train['Age'].median()

In [8]:
train['Age'] = train['Age'].fillna(impute_value)

In [9]:
test['Age'] = test['Age'].fillna(impute_value)

### 然后再添加了一列 IsFemale 作为'Sex'列的编码版本。

In [10]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)

In [11]:
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [12]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsFemale
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,28.0,1,2,W./C. 6607,23.4500,NaN,S,1
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0


### 然后决定一些模型变量并创建NumPy数组。

In [13]:
predictors = ['Pclass', 'IsFemale', 'Age']

In [14]:
X_train = train[predictors].to_numpy()

In [15]:
X_test = test[predictors].to_numpy()

In [16]:
y_train = train['Survived'].to_numpy()

In [17]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [18]:
y_train[:5]

array([0, 1, 1, 1, 0], dtype=int64)

# 3.创建模型：

### 使用scikit-learn的LogisticRegression模型并创建一个模型实例（该模型仅为实例，不代表是一个合适的模型，同时前面选取的特征也不一定是最正确的）。

In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
model = LogisticRegression()

### 我们可以使用模型的 fit 方法在训练数据上拟合模型。

In [21]:
model.fit(X_train, y_train)

LogisticRegression()

### 现在，我们可以使用model.predict对测试数据集进行预测。

In [22]:
y_predict = model.predict(X_test)

In [23]:
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

### 如果你有测试数据集的真实值，你可以计算出准确率百分比或其他一些误差指标。

In [24]:
# (y_true == y_predict).mean()

# 4.交叉验证法：

In [25]:
from sklearn.linear_model import LogisticRegressionCV

In [26]:
model_cv = LogisticRegressionCV(Cs=10)

In [27]:
model_cv.fit(X_train, y_train)

LogisticRegressionCV()

### 如果需要手工进行交叉验证，可以使用 cross_val_score 辅助函数，它用于处理数据拆分过程。例如，为了用练数据的四个非重叠分割进行交叉验证，可以采用如下方式。

In [28]:
from sklearn.model_selection import cross_val_score

In [29]:
model = LogisticRegression(C=10)

In [30]:
scores = cross_val_score(model, X_train, y_train, cv=4)

In [31]:
scores

array([0.77578475, 0.79820628, 0.77578475, 0.78828829])